In [ ]:
%pip install feast[ge]

In [ ]:
import pyarrow.parquet
import pandas as pd

from feast import FeatureView, Entity, FeatureStore, Field, BatchFeatureView
from feast.types import Float64, Int64
from feast.value_type import ValueType
from feast.data_format import ParquetFormat
from feast.on_demand_feature_view import on_demand_feature_view
from feast.infra.offline_stores.file_source import FileSource
from feast.infra.offline_stores.file import SavedDatasetFileStorage
from datetime import timedelta

import numpy as np

from feast.dqm.profilers.ge_profiler import ge_profiler

from great_expectations.core.expectation_suite import ExpectationSuite
from great_expectations.dataset import PandasDataset

from feast.dqm.errors import ValidationFailed

# Data Validation with Feast Saved Datasets & Great Expectations

The purpose of this notebook is to showcase the potential for data validation with Feast's Saved Dataset feature, harnessing the power of the Great Expectations validation engine.

## The Data

### trip_stats.parquet

Driver statistics for a taxi service. Each column corresponds to a driver's statistics for the day, given that they had at least one trip. The columns are: `'taxi_id', 'day', 'total_miles_travelled', 'total_trip_seconds', 'total_earned', 'trip_count'`

### trip_stats_featurized.parquet

This is my attempt at featurizing the trip stats data into something a logistic regression model could use. Please bear with me, I have zero data science experience, so these features are arbitrarily chosen without regard and the code to get there (in demo.ipynb) is NOT ideal lol. Anyway, the columns are: `'taxi_id', 'day', 'trip_count_1_to_10', 'trip_count_11_to_25', 'trip_count_26_to_50', 'trip_count_51_to_100', 'trip_count_101_or_higher', 'total_miles_0_to_10', 'total_miles_10_to_25', 'total_miles_25_to_50', 'total_miles_50_to_100', 'total_miles_100_to_250', 'total_miles_250_to_500', 'total_miles_500_to_1000', 'total_miles_1000_or_higher', 'seconds_0_3600', 'seconds_3600_to_7200', 'seconds_7200_to_14400', 'seconds_14400_to_28800', 'seconds_28800_to_43200', 'seconds_43200_or_higher', 'earned_0_to_50', 'earned_50_to_100', 'earned_100_to_250', 'earned_250_to_500', 'earned_500_to_1000', 'earned_1000_or_higher'`

## The Goal

The goal is to show how Feast & Great Expectations can be used to:

1. Validate the structure and content of pre-featurization data. This will break the assumption that Feast stores only feature data, but it will show the power of Great Expectations.
2. Track the drift of distributions of features over time.

